## 195 PoC: Koppla SAT-leden till Naturvårdsverkets “Skyddad natur” + maskinöversatta föreskrifter

* issue [#195](https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/195)
* den här Noteboken 195_Skyddadnatur_translate.ipynb

[Reservatsnamn]  
📜 Föreskrifter: [SV]  
🌍 Maskinöversatt: [DA] [NN] [EN] [FR] [ZH] [AR]


In [1]:
import time

from datetime import datetime

now = datetime.now()
timestamp = now.timestamp()

start_time = time.time()
print("Start:", datetime.now().strftime("%Y-%m-%d %H:%M:%S"))

Start: 2025-09-29 12:28:17


In [2]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

import json, sys, time, re
from io import BytesIO, StringIO
from urllib.parse import urlencode
import requests
from urllib.parse import urlparse, unquote

# --- (valfritt) geopandas för "svårare" P3896-källor (WFS/ZIP) ---
try:
    import geopandas as gpd        # pip install geopandas
except Exception:
    gpd = None

WIKIDATA_SPARQL = "https://query.wikidata.org/sparql"
UA = {"User-Agent": "SAT-pipeline/1.0 (contact: your-email@example.com)"}

# Fil att skriva
OUTFILE = "SAT_reserves_translations.geojson"

# Språklistor
RESIDENT_LANGS = ["sv","en","ar","fi","so","fa","ckb","ti","pl","tr","es"]
TOURIST_LANGS  = ["nb","nn","da","fi","de","nl","en","fr","es","it","zh","ja","pl","ru"]

def google_translate_url(src_sv_url: str, tl: str) -> str:
    base = "https://translate.google.com/translate"
    q = {"hl": "sv", "sl": "sv", "tl": tl, "u": src_sv_url}
    return f"{base}?{urlencode(q)}"

def fetch_sparql():
    q = """
    SELECT ?res ?resLabel ?foreskrift ?shape ?coord WHERE {
      wd:Q131318799 wdt:P3018 ?res .
      OPTIONAL { ?res wdt:P856 ?foreskrift }
      OPTIONAL { ?res wdt:P3896 ?shape }
      OPTIONAL { ?res wdt:P625 ?coord }
      SERVICE wikibase:label { bd:serviceParam wikibase:language "sv,en". }
    }
    """
    r = requests.get(WIKIDATA_SPARQL, params={"query": q, "format": "json"}, headers=UA, timeout=60)
    r.raise_for_status()
    return r.json()["results"]["bindings"]

def is_commons_map(url: str) -> bool:
    # Commons Data namespace .map (GeoJSON)
    return ("commons.wikimedia.org" in url) and ("/data/main/Data:" in url or "action=raw" in url) and url.endswith(".map")

def fetch_geojson_from_url(url: str):
    try:
        r = requests.get(url, headers=UA, timeout=60)
        r.raise_for_status()
        gj = r.json()

        # Plocka ut själva datan om Commons .map har metadata-wrapper
        if "data" in gj:
            gj = gj["data"]

        # Dict med FeatureCollection
        if isinstance(gj, dict):
            if gj.get("type") == "FeatureCollection":
                return gj
            if gj.get("type") == "Feature":
                return {"type": "FeatureCollection", "features": [gj]}

        # Lista med features
        if isinstance(gj, list):
            return {"type": "FeatureCollection", "features": gj}

        return None
    except Exception as e:
        print("⚠️ error:", e, url)
        return None




def parse_point_wkt(wkt: str):
    # "Point(lon lat)" → (lon, lat)
    try:
        s = wkt.strip()
        s = s.replace("POINT", "Point")
        s = s[s.index("(")+1:s.rindex(")")]
        lon, lat = [float(x) for x in s.split()]
        return lon, lat
    except Exception:
        return None

def build_feature_from_point(lon: float, lat: float, props: dict) -> dict:
    return {
        "type": "Feature",
        "geometry": {"type": "Point", "coordinates": [lon, lat]},
        "properties": props,
    }

def attach_properties_to_all_features(gj: dict, props: dict) -> dict:
    out = {"type": "FeatureCollection", "features": []}
    for f in gj.get("features", []):
        g = f.get("geometry")
        if not g:
            continue
        p = f.get("properties", {}).copy()
        p.update(props)
        out["features"].append({"type": "Feature", "geometry": g, "properties": p})
    return out
    
import urllib.parse

from urllib.parse import urlparse, unquote

def commons_map_url(in_url: str) -> str:
    if not in_url:
        return None
    parsed = urlparse(in_url)

    # Extract path after /data/main/
    path = parsed.path.split("/data/main/")[-1]

    # Decode percent-encodings (e.g. %C3%B6 → ö)
    path = unquote(path)

    # Replace + with _  (Commons convention)
    path = path.replace("+", "_")
    return f"https://commons.wikimedia.org/w/index.php?title={path}&action=raw"



def main():
    rows = fetch_sparql()
    features = []
    for b in rows:
        label = b.get("resLabel", {}).get("value") or "(namnlös)"
        fores = b.get("foreskrift", {}).get("value")
        shape = b.get("shape", {}).get("value")
        coord = b.get("coord", {}).get("value")
    
        if not shape:
            print(f"{label}: ingen P3896")
            continue
    
        url = commons_map_url(shape)
        gj = fetch_geojson_from_url(url)
        # Gör en dict för alla språk: trans_<språkkod>
        trans_props = {}
        if fores:
            for tl in sorted(set(RESIDENT_LANGS + TOURIST_LANGS)):
                trans_props[f"trans_{tl}"] = google_translate_url(fores, tl)
        
        props = {
            "name": label,
            "foreskrift_sv": fores,
            **trans_props,  # <-- plattar ut översättningar
            "source": "Wikidata P3018/P856/P3896; Commons/GeoJSON där tillgängligt",
        }
    
        added = False
        if gj:  # använd gj från fetch_geojson_from_url(url)
            gj_props = attach_properties_to_all_features(gj, props)
            features.extend(gj_props["features"])
            added = True
        else:
            print(f"{label}: ❌ ingen GeoJSON från {url}")
    
        if not added:
            if coord:
                pt = parse_point_wkt(coord)
                if pt:
                    lon, lat = pt
                else:
                    lon, lat = 18.06, 59.33
            else:
                lon, lat = 18.06, 59.33
            features.append(build_feature_from_point(lon, lat, props))

        # Hövlig paus så vi inte spammar endpoints
        time.sleep(0.2)

    fc = {"type": "FeatureCollection", "features": features}
    # skriv
    with open(OUTFILE, "w", encoding="utf-8") as f:
        json.dump(fc, f, ensure_ascii=False, indent=2)

    print(f"✅ Skrev {OUTFILE} med {len(features)} features.")

if __name__ == "__main__":
    main()
    import geopandas as gpd
    gdf = gpd.read_file("SAT_reserves_translations.geojson")
    print(gdf.geom_type.value_counts())

✅ Skrev SAT_reserves_translations.geojson med 12 features.
MultiPolygon    10
Polygon          2
Name: count, dtype: int64


### Map 

In [3]:
def add_about_box(
    m,
    issue_number: int,
    map_name: str,
    created_date: str | None = None,
    repo: str = "salgo60/Stockholm_Archipelago_Trail",
    collapsed: bool = False,
):
    """
    Notebook/Edge-säker About-box med string.Template ($-placeholder).
    """
    if created_date is None:
        created_date = datetime.now().strftime("%Y-%m-%d %H:%M")

    map_dom_id = m.get_name()
    box_id     = f"sat-about-{map_dom_id}"
    header_id  = f"{box_id}-hdr"
    issue_url  = f"https://github.com/{repo}/issues/{issue_number}"

    links = [
        ("▶️ video about the map", "https://youtu.be/4MmD1EctW2Y"),            
        ("SAT Dashboard", "https://raw.githack.com/salgo60/Stockholm_Archipelago_Trail/main/notebook/output/SAT_ALL_IN_ONE_142_3_dashboard_latest.html"),
        ("Project repo issues", "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues?q=is%3Aissue"),
        ("Trail on OSM (rel 19012437)", "https://www.openstreetmap.org/relation/19012437"),
        ("Trail on Wikicommons", "https://commons.wikimedia.org/wiki/Category:Stockholm_Archipelago_Trail"),
        ("Official page", "https://stockholmarchipelagotrail.com/"),
        ("Unofficial FB group", "https://www.facebook.com/groups/2875020699552247"),
        ("Visit Sweden", "https://traveltrade.visitsweden.com/plan/news-sweden/Stockholm-Archipelago-Trail/"),
    ]
    links_html = "".join(
        f'<div><a href="{_html.escape(u)}" target="_blank" style="text-decoration:none;">🔗 {_html.escape(t)}</a></div>'
        for t, u in links
    )
    collapsed_class = "sat-about-collapsed" if collapsed else ""

    tpl = Template(r"""
<style>
  .sat-about { position: fixed; z-index: 10000; background: rgba(255,255,255,0.97);
    border: 2px solid #666; border-radius: 10px; box-shadow: 0 2px 6px rgba(0,0,0,0.25);
    font: 12px/1.35 -apple-system, system-ui, Segoe UI, Roboto, Helvetica, Arial, sans-serif;
    pointer-events: auto; min-width: 240px; max-width: 320px; }
  .sat-about-header { cursor: pointer; padding: 8px 10px; font-weight: 700;
    display: flex; align-items: center; gap: 6px; user-select: none;
    background: rgba(248,248,248,.9); border-bottom: 1px solid #e5e7eb; }
  .sat-about-body { padding: 8px 10px 10px 10px; }
  .sat-about-collapsed .sat-about-body { display: none; }
  .sat-chevron { margin-left: auto; transition: transform .15s ease-in-out; }
  .sat-about-collapsed .sat-chevron { transform: rotate(-90deg); }
  .sat-links { margin-top: 6px; padding-top: 6px; border-top: 1px solid #e5e7eb; }
  @media (max-width: 640px) { .sat-about { font-size: 11px; min-width: 200px; max-width: 260px; } }
</style>

<div id="$box_id" class="sat-about $collapsed_class">
  <div id="$header_id" class="sat-about-header" title="Click to collapse/expand">
    <span>ℹ️ About</span>
    <span class="sat-chevron">▸</span>
  </div>
  <div class="sat-about-body">
    <div style="font-weight:700;margin-bottom:4px;">Stockholm Archipelago Trail Map</div>
    <div>Issue: <a href="$issue_url" target="_blank">#$issue_number</a>&nbsp;&nbsp; Map: $map_name</div>
    <div>Created: $created_date</div>
    <div class="sat-links">$links_html</div>
  </div>
</div>

<script>
(function(){
  var mapId = "$map_dom_id";
  var boxId = "$box_id";
  var hdrId = "$header_id";
  var storageKey = "satAboutCollapsed_" + "$map_dom_id" + "_#$issue_number";

  function setCollapsed(box, collapsed) {
    var body = box.querySelector(".sat-about-body");
    if (collapsed) { box.classList.add("sat-about-collapsed"); if (body) body.style.display = "none"; }
    else { box.classList.remove("sat-about-collapsed"); if (body) body.style.display = "block"; }
    try { localStorage.setItem(storageKey, collapsed ? "1" : "0"); } catch(e) {}
  }

  function placeBox(mapEl, box) {
    var zoom = mapEl.querySelector(".leaflet-control-zoom");
    var mr   = mapEl.getBoundingClientRect();
    var top = 10, left = 10;
    if (zoom) {
      var zr = zoom.getBoundingClientRect();
      top  = (zr.bottom - mr.top) + 8;
      left = (zr.left   - mr.left) + zr.width + 8;
    }
    box.style.top  = top  + "px";
    box.style.left = left + "px";
  }

  function init(tries) {
    tries = tries || 0;
    var mapEl = document.getElementById(mapId);
    var box   = document.getElementById(boxId);
    var hdr   = document.getElementById(hdrId);
    if (!mapEl || !box || !hdr) { if (tries < 60) return setTimeout(function(){ init(tries+1); }, 120); return; }

    try {
      var stored = localStorage.getItem(storageKey);
      if (stored === "1") setCollapsed(box, true);
      if (stored === "0") setCollapsed(box, false);
    } catch(e) {}

    hdr.addEventListener("click", function(e){
      e.stopPropagation();
      var willCollapse = !box.classList.contains("sat-about-collapsed");
      setCollapsed(box, willCollapse);
    });

    function doPlace(){ placeBox(mapEl, box); }
    var placeTries = 0, iv = setInterval(function(){ doPlace(); if (++placeTries > 25) clearInterval(iv); }, 120);
    window.addEventListener("resize", doPlace);
    requestAnimationFrame(doPlace);
  }

  if (document.readyState === "loading") { document.addEventListener("DOMContentLoaded", function(){ init(0); }); }
  else { init(0); }
})();
</script>
""")

    html_snippet = tpl.substitute(
        box_id=box_id,
        header_id=header_id,
        collapsed_class=collapsed_class,
        issue_url=issue_url,
        issue_number=str(issue_number),
        map_name=_html.escape(map_name),
        created_date=_html.escape(created_date),
        links_html=links_html,
        map_dom_id=map_dom_id,
    )
    m.get_root().html.add_child(folium.Element(html_snippet))


In [4]:
import shutil
from pathlib import Path

def _to_latest_name(filename: str | Path) -> Path:
    p = Path(filename)
    stem = p.stem
    new_stem = re.sub(r'_(?:20\d{6})(?:_\d{4})?$', '', stem)
    return p.with_name(new_stem + "_latest.html")

def save_with_latest(map_object, filename: str | Path):
    map_object.save(str(filename))
    latest_path = _to_latest_name(filename)
    shutil.copyfile(filename, latest_path)
    print(f"Saved: {filename}\nUpdated: {latest_path}")


In [7]:
import html as _html 
import json
import folium
import os
import pandas as pd
from datetime import datetime
from string import Template

# --- Language groups for reserves ---
RESIDENT_LANGS = ["sv","en","ar","fi","so","fa","ckb","ti","pl","tr","es"] 
TOURIST_LANGS  = ["nb","nn","da","fi","de","nl","en","fr","es","it","zh","ja","pl","ru"]

LANG_LABELS_EN = {
    "sv": "Swedish",
    "en": "English",
    "ar": "Arabic",
    "fi": "Finnish",
    "so": "Somali",
    "fa": "Persian",
    "ckb": "Kurdish (Sorani)",
    "ti": "Tigrinya",
    "pl": "Polish",
    "tr": "Turkish",
    "es": "Spanish",
    "nb": "Norwegian (Bokmål)",
    "nn": "Norwegian (Nynorsk)",
    "da": "Danish",
    "de": "German",
    "nl": "Dutch",
    "fr": "French",
    "it": "Italian",
    "zh": "Chinese",
    "ja": "Japanese",
    "ru": "Russian",
}

# --- Create map ---
m = folium.Map(location=[59.32, 18.06], zoom_start=9, tiles="OpenStreetMap")

# --- Add feature groups for toggleable layers ---
trail_layer = folium.FeatureGroup(name="SAT Trail", show=True)
reserves_layer = folium.FeatureGroup(name="Nature Reserves", show=True)


# --- Load trail GeoJSON ---
with open("SAT_full.geojson", encoding="utf-8") as f:
    sat_geojson = json.load(f)

# --- Load reserves GeoJSON ---
with open("SAT_reserves_translations.geojson", encoding="utf-8") as f:
    reserves_geojson = json.load(f)
# --- Feature groups ---
trail_layer = folium.FeatureGroup(name="SAT Trail", show=True)
reserves_layer = folium.FeatureGroup(name="Nature Reserves", show=True)

# --- Popup for trail sections ---
def make_trail_popup(props: dict) -> str:
    name = props.get("Labelen") or props.get("Labelsv") or "SAT Trail"
    web_sv = props.get("website_sv")
    web_en = props.get("website_en")
    commons_cat = props.get("P373")
    source = props.get("source", "Wikidata/Commons")

    links = []

    if web_sv:
        links.append(f'🇸🇪 <a href="{web_sv}" target="_blank">Official Swedish page</a>')
    if web_en:
        links.append(f'🇬🇧 <a href="{web_en}" target="_blank">Official English page</a>')
    links.append(f'▶️ <a href="https://youtu.be/4MmD1EctW2Y" target="_blank">video about the map</a>')

    commons_links = ""
    if commons_cat:
        cat_enc = commons_cat.replace(" ", "_")
        commons_url = f"https://commons.wikimedia.org/wiki/Category:{cat_enc}"
        map_url = f"https://wikimap.toolforge.org/?cat={cat_enc}&subcats=true&subcatdepth=3&cluster=false"
        commons_links = f"""
        <p>
        📸 <a href="{commons_url}" target="_blank">Wikicommons pictures</a> /
        🗺️ <a href="{map_url}" target="_blank">Map</a>
        </p>
        """

    translation_section = ""
    if web_en:
        def google_translate_url(url, tl):
            from urllib.parse import urlencode
            base = "https://translate.google.com/translate"
            q = {"hl": "en", "sl": "en", "tl": tl, "u": url}
            return f"{base}?{urlencode(q)}"

        res_links = " / ".join(
            f'<a href="{google_translate_url(web_en, code)}" target="_blank">{LANG_LABELS_EN.get(code, code)}</a>'
            for code in RESIDENT_LANGS if code != "en"
        )
        tour_links = " / ".join(
            f'<a href="{google_translate_url(web_en, code)}" target="_blank">{LANG_LABELS_EN.get(code, code)}</a>'
            for code in TOURIST_LANGS if code != "en"
        )

        translation_section = f"""
        <details>
          <summary>🌍 Translations of the English official page (machine translated)</summary>
          <p><strong>⚠️ Note:</strong> These translations are machine generated.</p>
          <h4>Languages spoken in Sweden</h4>
          <p>{res_links}</p>
          <h4>Common tourist languages</h4>
          <p>{tour_links}</p>
        </details>
        """

    return f"""
    <h3>{name}</h3>
    <p>{"<br>".join(links)}</p>
    {commons_links}
    {translation_section}
    <p><em>Source: {source}</em></p>
    """

# --- Popup for reserves ---
def make_reserve_popup(props: dict) -> str:
    name = props.get("name", "Unknown reserve")
    foreskrift = props.get("foreskrift_sv")
    source = props.get("source", "Wikidata/Commons")

    foreskrift_link = (
        f"📄 <a href='{foreskrift}' target='_blank'>Swedish regulation (original)</a><br>"
        if foreskrift else ""
    )

    def links_for_group(lang_list):
        return " / ".join(
            f'<a href="{props.get(f"trans_{code}")}" target="_blank">{LANG_LABELS_EN.get(code, code)}</a>'
            for code in lang_list if props.get(f"trans_{code}")
        )

    res_html = links_for_group(RESIDENT_LANGS) or "No translations available"
    tour_html = links_for_group(TOURIST_LANGS) or "No translations available"

    return f"""
    <h3>{name}</h3>
    {foreskrift_link}
    <p><strong>⚠️ Note:</strong> The texts below are machine translated 
    - ▶️ <a href="https://youtu.be/4MmD1EctW2Y" target="_blank">video</a>.</p>
    <h4>🌍 Languages spoken in Sweden</h4>
    <p>{res_html}</p>
    <h4>🧳 Common tourist languages</h4>
    <p>{tour_html}</p>
    <p><em>Source: {source}</em></p>
    """

# --- Add trail features ---
for feat in sat_geojson["features"]:
    props = feat.get("properties", {})
    folium.GeoJson(
        feat,
        style_function=lambda x: {"color": "green", "weight": 3},
        tooltip=props.get("Labelen") or props.get("Labelsv") or "SAT Trail",
        popup=folium.Popup(make_trail_popup(props), max_width=400),
    ).add_to(trail_layer)

# --- Add reserve features ---
for feat in reserves_geojson["features"]:
    props = feat.get("properties", {})
    folium.GeoJson(
        feat,
        style_function=lambda x: {"color": "blue", "weight": 1, "fillOpacity": 0.3},
        tooltip=props.get("name", ""),
        popup=folium.Popup(make_reserve_popup(props), max_width=400),
    ).add_to(reserves_layer)

# --- Add groups and controls ---
trail_layer.add_to(m)
reserves_layer.add_to(m)
folium.LayerControl(collapsed=False).add_to(m)


# --- Popup for reserves ---
def make_reserve_popup(props: dict) -> str:
    name = props.get("name", "Unknown reserve")
    foreskrift = props.get("foreskrift_sv")
    source = props.get("source", "Wikidata/Commons")

    foreskrift_link = (
        f"📄 <a href='{foreskrift}' target='_blank'>Swedish regulation (original)</a><br>"
        if foreskrift else ""
    )

    def links_for_group(lang_list):
        return " / ".join(
            f'<a href="{props.get(f"trans_{code}")}" target="_blank">{LANG_LABELS_EN.get(code, code)}</a>'
            for code in lang_list if props.get(f"trans_{code}")
        )

    res_html = links_for_group(RESIDENT_LANGS) or "No translations available"
    tour_html = links_for_group(TOURIST_LANGS) or "No translations available"

    return f"""
    <h3>{name}</h3>
    {foreskrift_link}
    <p><strong>⚠️ Note:</strong> The texts below are machine translated.
    Only the Swedish original text is legally binding.</p>

    <h4>🌍 Languages spoken in Sweden</h4>
    <p>{res_html}</p>

    <h4>🧳 Common tourist languages</h4>
    <p>{tour_html}</p>

    <p><em>Source: {source}</em></p>
    """
    
# --- Save helper ---
def save_with_latest(m, path: str):
    m.save(path)
    latest_path = path.replace(
        pd.Timestamp.now().strftime("%Y%m%d_%H%M"), "latest"
    )
    m.save(latest_path)
    print(f"✅ Saved {path} and {latest_path}")

# --- Save map ---
PROJECT_NAME = "SAT_195_MachineTranslate"
OUTPUT_DIR = "./output"
os.makedirs(OUTPUT_DIR, exist_ok=True)
stamp = pd.Timestamp.now().strftime("%Y%m%d_%H%M")
html_path = os.path.join(OUTPUT_DIR, f"{PROJECT_NAME}_{stamp}.html")

# Lägg till About-box
add_about_box(m, issue_number=195, map_name="Machine Translate Regulations (English UI)")
save_with_latest(m, html_path)




✅ Saved ./output/SAT_195_MachineTranslate_20250929_1229.html and ./output/SAT_195_MachineTranslate_latest.html


In [8]:
 # End timer and calculate duration
end_time = time.time()
elapsed_time = end_time - start_time# Bygg audit-lager för den här etappen

# Print current date and total time
print("Date:", datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
minutes, seconds = divmod(elapsed_time, 60)
print("Total time elapsed: {:02.0f} minutes {:05.2f} seconds".format(minutes, seconds))


Date: 2025-09-29 12:29:25
Total time elapsed: 01 minutes 08.44 seconds
